In [1]:
dimension = 'color'

In [2]:
import json
json_filepath = '../Human_anno/{}.json'.format(dimension)
with open(json_filepath,'r') as f:
    oc = json.load(f)

In [3]:
def rank_basescore(base_score, draw_ratio):
    # 计算最大值和最小值的差值
    score_range = max(base_score) - min(base_score)
    # 计算平序的阈值
    draw_gap = draw_ratio * score_range

    # 对列表进行排序并保留原始索引
    indexed_scores = list(enumerate(base_score))
    indexed_scores.sort(key=lambda x: x[1])

    # 处理平序
    ranks = [0] * len(base_score)
    current_rank = 1
    for i in range(len(indexed_scores)):
        if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:
            ranks[indexed_scores[i][0]] = current_rank
        else:
            current_rank = i + 1
            ranks[indexed_scores[i][0]] = current_rank

    return ranks

In [4]:
models = ['cogvideox5b','gen3','kling','lavie','pika','show1','videocrafter2']
vbench_res = {}
for model in models:
    path = '../baseline/{}/grit/results_color_{}_eval_results.json'.format(dimension,model)
    with open(path,'r') as f:
        vbench_res[model] = json.load(f)

In [5]:
for model in models:
    for i in range(len(oc)):
        for j in range(len(vbench_res[model][dimension][1])):
            if oc[i]['videos'][model].split('/')[-1] == vbench_res[model][dimension][1][j]['video_path'].split('/')[-1]:
                oc[i]['baseline_score'][model] = vbench_res[model][dimension][1][j]['video_results']
                break
        if j == len(vbench_res[model][dimension][1])-1:
            oc[i]['baseline_score'][model] = 0


In [6]:
with open(json_filepath,'w') as f:
    json.dump(oc,f,indent=4)